In [91]:
import os,datetime
from PIL import Image,ExifTags
import re
import hashlib
from hachoir.parser import createParser
import warnings
from hachoir.metadata import extractMetadata
import piexif

In [92]:
def filename_to_date_via_format(file, formatt):
    result = datetime.datetime.strptime(file, formatt).strftime("%Y:%m:%d %H:%M:%S")
    return result

In [93]:
def creation_date_of_video(filename):
    
    # Special cases..
    #video-2012-12-29-18-40-59.mp4
    if re.search('video-[0-9]{4}-[0-9]{2}-[0-9]{2}-[0-9]{2}-[0-9]{2}-[0-9]{2}.mp4', filename) is not None:
        
        return filename_to_date_via_format(re.sub('.+video', 'video', filename), 'video-%Y-%m-%d-%H-%M-%S.mp4')
    
    parser = createParser(filename)
    metadata = extractMetadata(parser)
    return metadata.get('creation_date').strftime("%Y_%m_%d %H_%M_%S")

In [94]:
def replace_exif(file, datetime_object):
    
    if re.search('[mMpP]4$', file) is not None:
        return

    img = Image.open(file)

    try:
        # If the image already contains data, we only replace the relevant properties
        exif_dict = piexif.load(img.info['exif'])
     #   print(f"Exif load for file '{file}'' successful")
    except KeyError:
        # If the image has no Exif data, we create the relevant properties
     #   print(f"No Exif data for file '{file}', creating Exif data instead...")
        exif_dict = {}
        exif_dict["0th"] = {}
        exif_dict["Exif"] = {}

    # We now have a useful Exif dict, time to adjust the values
    d = datetime_object
    exif_dict["Exif"][36868] = d
    exif_dict["Exif"][36867] = d
    exif_dict["0th"][306] = d

    # Convert into bytes and dump into file
    exif_bytes = piexif.dump(exif_dict)
    piexif.insert(exif_bytes, file)
    print(f"SUCCESS: Exif data replacement for file '{file}' successful")

In [95]:
def creation_date_by_filename(filename, root):
    if re.search('\.([pngPNGjpgJPGEe]{3,4})$', filename) is None:
        return None

    try:
        cleanup = re.sub('\n', '', open(os.path.join(root, 'cleanup.txt'),'r').read())
        formatt = re.sub('\n', '', open(os.path.join(root, 'format.txt'),'r').read())

        name_tobeformatted = re.search(cleanup, filename).group()
        date = filename_to_date_via_format(name_tobeformatted, formatt)
        
        try:
            replace_exif(filename)
        except:
            print('Moving file but could not write exif data. ' + filename)
        return date
    except:
        return None

In [96]:
# Don't call this directly..
def get_exif_date_proxy(file, root):
    try:
        img = Image.open(file)
        try:
            return img._getexif()[36867]
        except:
            try: 
                return img._getexif()[306]
            except:    
                raise Error('No exif in it')
    except:
         try:
            return creation_date_of_video(file)
         except:
                try:
                    return creation_date_by_filename(file, root)
                except:
                    return None


        
def get_exif_date(file, root):
    result = get_exif_date_proxy(file, root)
    if  result is None:
        return None
    else:
        result = re.sub(':', '_', result)
        if re.search('[0-9]{4}_[0-9]{2}_[0-9]{2} [0-9]{2}_[0-9]{2}_[0-9]{2}', result) is None:
            return None
        else:
            dateparsed = datetime.datetime.strptime(result, "%Y_%m_%d %H_%M_%S").year
            if dateparsed < 1991 or dateparsed > 2025:
                print('Tarihten red: ' + file)
                return None
            return result


In [97]:
def rename_preserving_extension(file, name):
    filename = re.sub('^\.\/', '', str(file))
    correctName = name + re.search('\.([^\.]+)$', filename).group()
    correctName = re.sub('[^0-9A-Z-_\.a-z\/]', '', correctName)
    os.rename(file, correctName);

In [98]:
def exif_create_folder_for_date(file, root):
    result = get_exif_date(file, root)
    result = re.sub(':', '_', result)
    result = re.sub('.{9}$', '', result)
    year = re.search('[0-9]{4}', result).group()
    try:
        os.mkdir(year)
    except:
        None
    try:
        os.mkdir(year + '/' + result)
    except:
        None
    return year + '/' + result


In [99]:
def exif_get_hour_of_creation(file, root):
    result = get_exif_date(file, root)
    result = re.sub(':', '_', result)
    result = re.sub('^.{11}', '', result)
    return result

In [100]:
def md5sum(filename):
    md5_hash = hashlib.md5()
    with open(filename,"rb") as f:
        # Read and update hash in chunks of 4K
        for byte_block in iter(lambda: f.read(4096),b""):
            md5_hash.update(byte_block)
        return re.search('^.{5}', md5_hash.hexdigest()).group()

In [104]:
directory = os.fsencode('.')
    
for root,d_names,f_names in os.walk('.'):
    if re.search('^(\.\/[0-9]{4}|\.\/[^\.]*\.)', str(root)) is None:
        print ("processing path: " + root)
        for f in f_names:
            file = os.path.join(root, f)
        
            if get_exif_date(file, root) is None:
                None # Skip
            else:
                folder = exif_create_folder_for_date(file, root)
                newname = folder + '/' + exif_get_hour_of_creation(file, root) + '_' + md5sum(file)
                rename_preserving_extension(file, newname)
                print(file + " -----> " + newname)
    else:
        print("ignored path: " + root)
print('end')

processing path: .


[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [/exif/content] [Autofix] Fix parser error: stop parser, found unparsed segment: start 80176, length 16


processing path: ./asd
processing path: ./asd/2021_11_11


[warn] [<JpegFile>] Only supporting Baseline & Extended Sequential JPEG images so far!


processing path: ./util
processing path: ./__pycache__


...

Moving file but could not write exif data. ./Rooll/20210227_133615431_iOS.jpg


[warn] [/exif/content] [Autofix] Fix parser error: stop parser, found unparsed segment: start 80176, length 16


Moving file but could not write exif data. ./Rooll/20210227_133615431_iOS.jpg


[warn] [/exif/content] [Autofix] Fix parser error: stop parser, found unparsed segment: start 80176, length 16


Moving file but could not write exif data. ./Rooll/20210227_133615431_iOS.jpg
./Rooll/20210227_133615431_iOS.jpg -----> 2021/2021_02_27/13_36_15_53776
ignored path: ./.ipynb_checkpoints
ignored path: ./.git
ignored path: ./.git/objects
ignored path: ./.git/objects/66
ignored path: ./.git/objects/3e
ignored path: ./.git/objects/6f
ignored path: ./.git/objects/69
ignored path: ./.git/objects/58
ignored path: ./.git/objects/0b
ignored path: ./.git/objects/5f
ignored path: ./.git/objects/05
ignored path: ./.git/objects/b5
ignored path: ./.git/objects/d7
ignored path: ./.git/objects/b3
ignored path: ./.git/objects/b4
ignored path: ./.git/objects/d6
ignored path: ./.git/objects/f3
ignored path: ./.git/objects/fe
ignored path: ./.git/objects/ed
ignored path: ./.git/objects/4b
ignored path: ./.git/objects/pack
ignored path: ./.git/objects/11
ignored path: ./.git/objects/6e
ignored path: ./.git/objects/36
ignored path: ./.git/objects/info
ignored path: ./.git/objects/62
ignored path: ./.git/obj